In [ ]:
!pip -q install langchain huggingface_hub transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.2/358.2 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00


## HuggingFace

There are two Hugging Face LLM wrappers, one for a local pipeline and one for a model hosted on Hugging Face Hub. Note that these wrappers only work for models that support the following tasks: text2text-generation, text-generation


In [ ]:
import os


os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

## Use the HuggingFaceHub

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=HuggingFaceHub(repo_id="google/flan-t5-xl", 
                                        model_kwargs={"temperature":0, 
                                                      "max_length":64}))

In [ ]:
question = "What is the capital of France?"

print(llm_chain.run(question))

Paris is the capital of France. The final answer: Paris.


In [ ]:
question = "What area is best for growing wine in France?"

print(llm_chain.run(question))

The best area for growing wine in France is the Loire Valley. The Loire Valley is located in the south of France. The area of France that is best for growing wine is the Loire Valley. The final answer: Loire Valley.


## BlenderBot

Doesn't work on the Hub

In [ ]:
blenderbot_chain = LLMChain(prompt=prompt, 
                     llm=HuggingFaceHub(repo_id="facebook/blenderbot-1B-distill", 
                                        model_kwargs={"temperature":0, 
                                                      "max_length":64}))

ValidationError: ignored

In [ ]:
# question = "What is the capital of France?"
# question = "What area is best for growing wine in France?"

# print(blenderbot_chain = LLMChain(prompt=prompt, 
# .run(question))

## With Local model from HF 

### Why would you want to use local mode?

- fine-tuned models
- GPU hosted etc
- some models only work locally

## T5-Flan - Encoder-Decoder

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'# go for a smaller model if you dont have the VRAM
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True)

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
print(local_llm('What is the capital of France? '))

paris


In [ ]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What is the capital of England?"

print(llm_chain.run(question))

The capital of England is London. London is the capital of England. So the answer is London.


## GPT2-medium - Decoder Only Model

microsoft/DialoGPT-large

In [ ]:
model_id = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What is the capital of France?"

print(llm_chain.run(question))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




The French Empire is made up of 8 provinces from the Atlantic to Pacific land masses.

From the Atlantic, France extends westward.

From the Pacific to Europe, France extends west.

From Europe to Asia, France spreads along the middle.

From Asia to the Mediterranean Sea, France follows the southward-southeast direction.

The coast line


## BlenderBot - Encoder-Decoder

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'facebook/blenderbot-1B-distill'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What area is best for growing wine in France?"

print(llm_chain.run(question))

 I'm not sure, but I do know that France is one of the largest producers of wine in the world.


## SentenceTransformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
hf.embed_query('this is an embedding')

[0.010657310485839844,
 -0.09967269003391266,
 -0.026967095211148262,
 0.06531771272420883,
 0.021005000919103622,
 0.04262349754571915,
 0.011534060351550579,
 -0.006229334976524115,
 0.05175827071070671,
 0.007306730840355158,
 0.021353425458073616,
 0.04269154742360115,
 0.023143811151385307,
 0.009952709078788757,
 0.05646306648850441,
 -0.06137978285551071,
 0.0527438223361969,
 0.024683939293026924,
 -0.013267776928842068,
 -0.007051161024719477,
 0.02665640041232109,
 -0.005913516040891409,
 0.004097455181181431,
 0.0384124331176281,
 -0.014230639673769474,
 0.023023542016744614,
 -0.007326607592403889,
 -0.03562537580728531,
 -0.017934124916791916,
 -0.013930189423263073,
 0.011977523565292358,
 -0.007365939673036337,
 0.024451466277241707,
 -0.06637256592512131,
 1.5677629789934144e-06,
 0.01821723021566868,
 0.00197486556135118,
 -0.018329503014683723,
 -0.014930644072592258,
 -0.00539350276812911,
 -0.011222291737794876,
 0.015792978927493095,
 -0.027141856029629707,
 -0.015

In [ ]:
hf.embed_documents(['this is an embedding','this another embedding'])

[[0.010657318867743015,
  -0.09967268258333206,
  -0.02696709893643856,
  0.06531770527362823,
  0.021004999056458473,
  0.042623501271009445,
  0.011534065939486027,
  -0.006229353602975607,
  0.0517583042383194,
  0.007306722458451986,
  0.021353380754590034,
  0.04269153252243996,
  0.023143835365772247,
  0.00995270162820816,
  0.056463032960891724,
  -0.06137979403138161,
  0.0527438260614872,
  0.024683943018317223,
  -0.013267838396131992,
  -0.007051167543977499,
  0.02665640041232109,
  -0.005913490429520607,
  0.004097461700439453,
  0.038412418216466904,
  -0.01423065084964037,
  0.023023542016744614,
  -0.007326596416532993,
  -0.03562536463141441,
  -0.017934132367372513,
  -0.013930188491940498,
  0.011977534741163254,
  -0.007365899626165628,
  0.024451464414596558,
  -0.06637255847454071,
  1.5677629789934144e-06,
  0.018217233940958977,
  0.0019748930353671312,
  -0.01832951232790947,
  -0.014930643141269684,
  -0.005393484607338905,
  -0.011222314089536667,
  0.015792

In [ ]:


hf = HuggingFaceHubEmbeddings(
    repo_id=model_name,
    task="feature-extraction",
    # huggingfacehub_api_token="my-api-key",
)